In [1]:
from datetime import date
import ast
import pandas as pd

pd.set_option('display.max_columns', None)
df = catalog.load('raw/openaire/researchproduct_collectedfrom_datasource#parquet')

[02/05/25 07:50:00] INFO     Loading data from                                                  ]8;id=265596;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=953067;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             raw/openaire/researchproduct_collectedfrom_datasource#parquet                         
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9491 entries, 0 to 40
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author              9460 non-null   object
 1   openAccessColor     0 non-null      object
 2   publiclyFunded      0 non-null      object
 3   type                9491 non-null   object
 4   language            9491 non-null   object
 5   country             1244 non-null   object
 6   subjects            9489 non-null   object
 7   mainTitle           9491 non-null   object
 8   description         9451 non-null   object
 9   publicationDate     9486 non-null   object
 10  format              9490 non-null   object
 11  bestAccessRight     6488 non-null   object
 12  id                  9491 non-null   object
 13  originalId          9491 non-null   object
 14  indicators          9486 non-null   object
 15  instance            9491 non-null   object
 16  isGreen             0 non-null 

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'country', 'subjects', 'mainTitle', 'description', 'publicationDate',
       'format', 'bestAccessRight', 'id', 'originalId', 'indicators',
       'instance', 'isGreen', 'isInDiamondJournal', 'publisher', 'source',
       'contributor', 'pid', 'contactPerson', 'embargoEndDate'],
      dtype='object')

In [8]:
df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)
#pd.json_normalize(df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None))



0     http://vocabularies.coar-repositories.org/docu...
1                                                  None
2                                                  None
3     http://vocabularies.coar-repositories.org/docu...
4     http://vocabularies.coar-repositories.org/docu...
                            ...                        
36    http://vocabularies.coar-repositories.org/docu...
37    http://vocabularies.coar-repositories.org/docu...
38                                                 None
39    http://vocabularies.coar-repositories.org/docu...
40                                                 None
Name: bestAccessRight, Length: 9491, dtype: object

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [5]:
def land_openaire_researchproduct_collectedfrom_datasource(df: pd.DataFrame)-> pd.DataFrame:

    df = df.convert_dtypes()

    expected_columns = [
        'author',
        'openAccessColor',
        'publiclyFunded',
        'type',
        'language',
        'country',
        'subjects',
        'mainTitle',
        'description',
        'publicationDate',
        'format',
        'bestAccessRight',
        'id',
        'originalId',
        'indicators',
        'instance',
        'isGreen',
        'isInDiamondJournal',
        'publisher',
        'source',
        'container',
        'contributor',
        'contactPerson',
        'coverage',
        'pid',
        'contactPerson',
        'embargoEndDate'
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df_researchproduct = df[expected_columns].copy()
    df.reset_index(drop=True, inplace=True)

    # language
    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df['bestAccessRight'].apply(lambda x: x['label'] if x else None)
    df_researchproduct['bestAccessRight_scheme'] = df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)

    ## indicators
    df_indicators = pd.json_normalize(df['indicators']).reset_index(drop=True)
    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']).reset_index(drop=True), df_indicators], axis=1)

    ## author
    df_researchproduct2author = df.explode('author').reset_index(drop=True)
    df_researchproduct2author = df_researchproduct2author[['id','author']]
    df_authors = pd.json_normalize(df_researchproduct2author['author']).reset_index(drop=True)
    df_researchproduct2author = pd.concat([df_researchproduct2author.drop(columns=['author']), df_authors], axis=1)

    ## originalId
    df_researchproduct2originalId = df.explode('originalId').reset_index(drop=True)
    df_researchproduct2originalId = df_researchproduct2originalId[['id','originalId']]

    # TODO country

    ## subjects
    df_researchproduct2subject = df.explode('subjects').reset_index(drop=True)
    df_researchproduct2subject = df_researchproduct2subject[['id','subjects']]
    df_subjects = pd.json_normalize(df_researchproduct2subject['subjects']).reset_index(drop=True)
    df_researchproduct2subject = pd.concat([df_researchproduct2subject.drop(columns=['subjects']), df_subjects], axis=1)

    # TODO description

    # TODO format

    # TODO instance

    # TODO source

    # TODO container
    
    # TODO contributor
    
    # TODO contactPerson

    # TODO coverage
    
    # pid
    df_researchproduct2pid = df.explode('pid').reset_index(drop=True)
    df_researchproduct2pid = df_researchproduct2pid[['id','pid']]
    df_pid = pd.json_normalize(df_researchproduct2pid['pid']).reset_index(drop=True)
    df_researchproduct2pid = pd.concat([df_researchproduct2pid.drop(columns=['pid']), df_pid], axis=1)

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=[
        'author', 'country', 'subjects','bestAccessRight', 
        'language', 'format', 'instance', 'originalId', 
        'container', 'source', 'pid', 'description',
        'contributor', 'contactPerson', 'coverage'
        ], inplace=True)

    df_researchproduct['load_datetime'] = date.today()
    df_researchproduct2originalId['load_datetime'] = date.today()
    df_researchproduct2author['load_datetime'] = date.today()
    df_researchproduct2subject['load_datetime'] = date.today()
    df_researchproduct2pid['load_datetime'] = date.today()

    return df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid


In [6]:
df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid = land_openaire_researchproduct_collectedfrom_datasource(df)

In [7]:
df_researchproduct

,openAccessColor,publiclyFunded,type,mainTitle,publicationDate,id,isGreen,isInDiamondJournal,publisher,embargoEndDate,language_code,language_label,bestAccessRight_label,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,load_datetime
0,None,None,other,Interfaces naturales como factor de progreso c...,2015-04-01,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,None,None,<NA>,<NA>,esl/spa,Spanish,OPEN,C5,0.0,0.0,C5,2.841867e-09,C5,9.624046e-10,C5,2025-01-30
1,None,None,other,Tendiendo redes sororas: experiencias feminist...,2018-10-01,dedup_wf_002::000a7274fa00c9a28ae36b95cbd45cd4,None,None,<NA>,<NA>,esl/spa,Spanish,None,C5,0.0,0.0,C5,2.841867e-09,C5,1.418572e-09,C5,2025-01-30
2,None,None,other,Taller de Intercambio gratuito de recursos bib...,2018-10-12,dedup_wf_002::001df40be58fbb6eda8fc11da7f6caed,None,None,<NA>,<NA>,esl/spa,Spanish,None,C5,0.0,0.0,C5,2.841867e-09,C5,1.418572e-09,C5,2025-01-30
3,None,None,other,ISO 9001 in software-developing VSEs assisted ...,2012-10-01,dedup_wf_002::005706c1d880d70e098a39d9b942b729,None,None,<NA>,<NA>,eng,English,OPEN,C5,0.0,0.0,C5,2.841867e-09,C5,6.801357e-10,C5,2025-01-30
4,None,None,other,Aplicación de la fluorescencia de rayos X al a...,1969-01-01,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,None,Laboratorio de Entrenamiento Multidisciplinari...,<NA>,Español,Español,OPEN,C5,0.0,0.0,C5,2.841867e-09,C5,2.224333e-10,C5,2025-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9486,None,None,other,El caso REPOTUR,2014-11-01,od______9441::ffc30ec382e705eab2029d822cc73169,None,None,<NA>,<NA>,Español,Español,None,C5,0.0,0.0,C5,2.841867e-09,C5,8.529256e-10,C5,2025-01-30
9487,None,None,other,"Informe científico de investigador: Croce, Mar...",2016-01-01,od______9441::ffd4f316f80842ebeb3c1c0c95e38113,None,None,<NA>,<NA>,Español,Español,OPEN,C5,0.0,0.0,C5,2.841867e-09,C5,1.090879e-09,C5,2025-01-30
9488,None,None,other,Engineering Concern-Sensitive Navigation Struc...,2010-01-01,od______9441::ffda4ef3bc369dd920acdcb2968f184e,None,None,<NA>,<NA>,Inglés,Inglés,RESTRICTED,C5,0.0,0.0,C5,2.841867e-09,C5,5.546645e-10,C5,2025-01-30
9489,None,None,other,Acta nº 1454,2017-04-11,od______9441::ffea7a280687320d91bd29f0f99aaf4a,None,None,<NA>,<NA>,Español,Español,OPEN,C5,0.0,0.0,C5,2.841867e-09,C5,1.241646e-09,C5,2025-01-30


In [8]:
df_researchproduct2originalId.head(5)

,id,originalId,load_datetime
0,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,50|od______1329::d16ffc72c35465a1ffa9dc493a5b9aee,2025-01-30
1,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,oai:sedici.unlp.edu.ar:10915/46429,2025-01-30
2,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,50|od______9441::dfd76241b15809f6bf71bbebbb554796,2025-01-30
3,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,oai:digital.cic.gba.gob.ar:11746/3398,2025-01-30
4,dedup_wf_002::000a7274fa00c9a28ae36b95cbd45cd4,50|od______1329::647983f5689fb2bc473bcfb24f25c7fd,2025-01-30


In [9]:
df_researchproduct2author.head(5)

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance,load_datetime
0,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Contreras, Víctor",Víctor,NaN,1.0,Contreras,NaN,NaN,NaN,2025-01-30
1,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Zavala, Gonzalo",Gonzalo,NaN,2.0,Zavala,NaN,NaN,NaN,2025-01-30
2,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Fernández, Daniel",Daniel,NaN,3.0,Fernández,NaN,NaN,NaN,2025-01-30
3,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Holc, Matías",Matías,NaN,4.0,Holc,NaN,NaN,NaN,2025-01-30
4,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Passarelli, Cinthya",Cinthya,NaN,5.0,Passarelli,NaN,NaN,NaN,2025-01-30


In [10]:
df_researchproduct2subject

,id,provenance,subject.scheme,subject.value,load_datetime
0,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,NaN,keyword,Trastorno Autístico,2025-01-30
1,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,NaN,keyword,Lenguaje,2025-01-30
2,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,NaN,keyword,Ciencias Informáticas,2025-01-30
3,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,NaN,keyword,"Artificial, augmented, and virtual realities",2025-01-30
4,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,NaN,keyword,interfaces naturales,2025-01-30
...,...,...,...,...,...
48288,od______9441::fff0cd99227527524b1436f07353b230,NaN,keyword,Biodiversity hotspot,2025-01-30
48289,od______9441::fff0cd99227527524b1436f07353b230,NaN,keyword,Unprotected areas,2025-01-30
48290,od______9441::fff0cd99227527524b1436f07353b230,NaN,keyword,"Zoología, Ornitología, Entomología, Etología",2025-01-30
48291,od______9441::fff0cd99227527524b1436f07353b230,NaN,SDG,15. Life on land,2025-01-30


In [11]:
df_researchproduct2pid[df_researchproduct2pid['value'].notna()]

,id,scheme,value,load_datetime
16,dedup_wf_002::047c3c4b2e3bc78a4fc497f59ef4d8a6,handle,11336/200553,2025-01-30
38,dedup_wf_002::094d4d195322b3fc342e5bfabf90fef9,handle,11336/134551,2025-01-30
42,dedup_wf_002::09898eace76d999feb1c0138a5dc4e5b,handle,11336/108182,2025-01-30
148,dedup_wf_002::1f5ccf71b5ca71ad58a2a1191586e517,handle,11336/236059,2025-01-30
152,dedup_wf_002::1fa4cc412ead642390309c6e7c9a6866,handle,11336/172230,2025-01-30
...,...,...,...,...
1299,dedup_wf_002::f365e26b0193481e7d477623014a9531,handle,11336/129920,2025-01-30
1363,dedup_wf_002::fe455fc45b6b14b57c2a81aa945af6c3,handle,11336/195873,2025-01-30
1372,doi_dedup___::2cdf681b3b09acf0d611c46578574e63,doi,10.25607/obp-842,2025-01-30
1373,doi_dedup___::2cdf681b3b09acf0d611c46578574e63,handle,10919/99727,2025-01-30
